## Analyze A/B Test Results


## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction
An A/b test run by an e-commerce website , the company has developed a new web page in order to try and increase the number of users . In the end of this test we will know if they should implement this new page, keep the old page, or perhaps run the experiment longer to make their decision.


<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)



a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df_R = pd.read_csv("../input/analyze-ab-test-results-notebook//ab_data.csv")
df = df_R.copy()
df.head()

b. Use the cell below to find the number of rows in the dataset.

In [ ]:
df.shape

c. The number of unique users in the dataset.

In [ ]:
df.user_id.nunique()

d. The proportion of users converted.

In [ ]:
len(df.query("converted == 1"))/(df.shape[0])

e. The number of times the `new_page` and `treatment` don't match.

In [ ]:
old_treat =  df.query("landing_page == 'old_page' and group == 'treatment'")
new_control = df.query("landing_page == 'new_page' and group == 'control'")
old_treat.shape[0] + new_control.shape[0]

f. Do any of the rows have missing values?

In [ ]:
df.isnull().sum()

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
n_important = new_control.append(old_treat).index
df2 = df.drop(n_important)

In [ ]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

a. How many unique **user_id**s are in **df2**?

In [ ]:
df2.user_id.nunique()

b. There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2.user_id.value_counts().max()

c. What is the row information for the repeat **user_id**? 

In [ ]:
df2.query("user_id == 773192")

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [ ]:
df2.drop_duplicates(subset=['user_id'] , inplace = True)



a. What is the probability of an individual converting regardless of the page they receive?

In [ ]:
convert_mean = df2.converted.mean()

b. Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
control_convert = df2.query("group == 'control'").converted.mean()
control_convert

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
treat_convert = df2.query("group == 'treatment'").converted.mean()
treat_convert

d. What is the probability that an individual received the new page?

In [ ]:
(df2.query("landing_page == 'new_page'").shape[0])/(df2.shape[0])

e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.

- The proportions of each page are balanced 
- These results suggest that there is not sufficient evidence to say that the treatment page leads to more conversions because the probability of conversion for the treatment group is less than that for the control group.

<a id='ab_test'></a>
### Part II - A/B Test



$$
our  \ hypothesis \\
H_0 : p_{new} - p_{old} \leq 0 \\
H_1 : p_{new} - p_{old} > 0
$$

Conversion rate for $p_{new}$ and $p_{new}$ under the null

In [ ]:
convert_mean

Find $n_{new}$

In [ ]:
n_new = df2.query('landing_page == "new_page"').shape[0]
n_new

Find $n_{old}$

In [ ]:
n_old = df2.query('landing_page == "old_page"').shape[0]
n_old

a. What is the **conversion rate** for $p_{new}$ under the null? 

In [ ]:
n_converted = len(df2.query("converted == 1 "))
n_con_new = len(df2.query("converted == 1 & landing_page == 'new_page' "))
n_con_old = len(df2.query("converted == 1 & landing_page == 'old_page' "))
n_con_new / n_converted 

b. What is the **conversion rate** for $p_{old}$ under the null? <br><br>

In [ ]:
n_con_old / n_converted

c. What is $n_{new}$, the number of individuals in the treatment group?

In [ ]:
len(df2.query("group == 'treatment' "))

d. What is $n_{old}$, the number of individuals in the control group?

In [ ]:
len(df2.query("group == 'control' "))

e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.choice([0, 1], size=n_old, p=[(1 - convert_mean), convert_mean])
new_page_converted

f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.choice([0, 1], size=n_new, p=[(1 - convert_mean), convert_mean])
old_page_converted

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
new_page_converted.mean() - old_page_converted.mean()

h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called **p_diffs**.

In [ ]:
p_diffs = []

for i in range(10000):
    new_page_converted = np.random.choice([0, 1], size=n_new, p=[(1 - convert_mean), convert_mean])
    old_page_converted = np.random.choice([0, 1], size=n_old, p=[(1 - convert_mean), convert_mean])
    p_diffs.append(new_page_converted.mean() - old_page_converted.mean())
    
p_diffs = np.asarray(p_diffs)

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
plt.hist(p_diffs)
plt.title("Differences in Conversion Rates", fontsize=16)
plt.xlabel("\n Difference in Probability", fontsize=12)
plt.ylabel("\n users", fontsize=12)
plt.axvline(treat_convert - control_convert, color='r');

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
obs_diff = treat_convert - control_convert

(p_diffs > obs_diff).mean()

k. Please explain using the vocabulary you've learned in this course what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

## results
### we would fail to reject the nulls becauese :
- The p-value calculated is 0.9065 
- This is far greater than the typical  𝛼  level of 0.05 in business studies

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = df2.query('group == "control" & converted == 1')['converted'].count()
convert_new = df2.query('group == "treatment" & converted == 1')['converted'].count()
n_old = df2.query('landing_page == "old_page"').shape[0]
n_new = df2.query('landing_page == "new_page"').shape[0]

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/) is a helpful link on using the built in.

In [ ]:
sm.stats.proportions_ztest([convert_new, convert_old], [n_new, n_old], alternative='larger')

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

- The calculated values align with those obtained during the bootstrapped hypothesis testing.

<a id='regression'></a>
### Part III - A regression approach
a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

- Logistic Regression 

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df2['intercept'] = 1 
df2[['new_page', 'old_page']] = pd.get_dummies(df2['landing_page'])
df2.head()

c. Use **statsmodels** to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part **b.** to predict whether or not an individual converts. 

In [ ]:
log_mod = sm.Logit(df2['converted'], df2[['intercept', 'new_page']])
result = log_mod.fit()

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
# Workaround for known bug with .summary() with updated scipy
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

result.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in **Part II**?

- The p-value (0.190) here remains above an  𝛼  level of 0.05, So We will still reject the null

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

In [ ]:
#Calculate area of lower tail
p_lower = (p_diffs < obs_diff).mean()

#Calculate area of upper tail
upper = p_diffs.mean() - obs_diff
p_upper = (p_diffs > upper).mean()

#Calculate total tail area
p_lower + p_upper

- Adding a relevant variable can prevent bias in the estimate of the other regression coefficient but can also increase variance of other regression coefficient.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in

In [ ]:
countries_df = pd.read_csv('../input/countries/countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df_new.head()

In [ ]:
# Creatint dummy variables
df_new[['CA', 'UK', 'US']] = pd.get_dummies(df_new['country'])
df_new.head()

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
### Fit Your Linear Model And Obtain the Results
log_mod = sm.Logit(df_new['converted'], df_new[['intercept', 'CA', 'UK']])
result = log_mod.fit()
result.summary()

## insights 
- the p-values for the countries are still above a 0.05 $\alpha$ level so we fail to reject the null 

In [ ]:
df_new['CA_page'] = df_new['CA'] * df_new['new_page']
df_new['UK_page'] = df_new['UK'] * df_new['new_page']
df_new.head()

In [ ]:
log_mod = sm.Logit(df_new['converted'], df_new[['intercept', 'new_page', 'CA', 'UK', 'CA_page', 'UK_page']])
result = log_mod.fit()
result.summary()

## insights 
- None of the variables have significant p-values
- failing to reject the null 
- we don't have sufficient evidence to say that the new page results in more conversions than the old page. 